#### This IPython notebook explores the data for preprocessing. The final data preprocessing will be done in the script file "Data_Preprocess.py".

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r'D:\ML_Projects\AI_Tech_ChatBot\Data\UbuntuDialogueText.csv')

In [5]:
df.head(10)

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
5,3,64545.tsv,2009-08-01T06:22:00.000Z,richardcavell,mechtech,"it's a wikipedia term. There is Prodigy, PRDI..."
6,3,98758.tsv,2009-05-20T07:58:00.000Z,C-00000100,Severity1,what is bartek trying to do?
7,3,98758.tsv,2009-05-20T08:01:00.000Z,C-00000100,Severity1,is he trying to play online video?
8,3,98758.tsv,2009-05-20T08:04:00.000Z,C-00000100,Severity1,"bartek go here, if this doesn't play then it g..."
9,3,86303.tsv,2010-08-24T19:56:00.000Z,pozic,Somelauw,the algorithms from aptitude were ported to ap...


In [5]:
df.shape

(1038324, 6)

In [17]:
df['from'].unique()

array(['bad_image', 'lordleemo', 'mechtech', ..., 'torias', 'alexs',
       'lokpest_'], dtype=object)

In [18]:
df['from'].nunique()

164083

In [6]:
df['to'].unique()

array([nan, 'bad_image', 'mechtech', ..., 'torias', 'alexs', 'lokpest_'],
      dtype=object)

In [16]:
df['to'].nunique()

149488

In [19]:
df['dialogueID'].unique()

array(['126125.tsv', '64545.tsv', '98758.tsv', ..., '19585.tsv',
       '51506.tsv', '99669.tsv'], dtype=object)

In [20]:
df['dialogueID'].nunique()

346108